In [20]:

from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPool2D, Dropout, Flatten
from keras.layers.convolutional import MaxPooling2D
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.utils import class_weight
from scipy import signal
from glob import glob
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [21]:
# constants:
SUBSAMPLE = 100  # scales the data by this factor can be replaced by divding the steps per epoch by batch size
COLUMNS = ['HandStart', 'FirstDigitTouch',
        'BothStartLoadPhase', 'LiftOff',
        'Replace', 'BothReleased']
SUBJECTS = range(1, 2)
N_LABELS = 6

# Below are the paths to the data. Please pay attention to the % and * signs, these are needed in the loop.
TRAIN_DATA_PATH = 'C:/Users/Sebastiaan/Desktop/Programming/MachineLearning/Datasets/EEG/train/subj%d_series*_data.csv'
#The path below is for the test data used for a kaggle submission. This is not very relevant to our project.
# TEST_DATA_PATH = 'C:/Users/Sebastiaan/Desktop/Programming/MachineLearning/Datasets/EEG/test/subj%d_series*_data.csv'
# TRAIN_DATA_PATH = 'C:/Users/bas/Documents/MachineLearning/train/subj%d_series*_data.csv' #path on my laptop
# TEST_DATA_PATH =  'C:/Users/bas/Documents/MachineLearning/test/subj%d_series*_data.csv' 

SUBMISSION_FOLDER = 'C:/Users/Sebastiaan/Desktop/Programming/MachineLearning/'
SUBMISSION_NAME = 'subbmision_vu_48_sub_pca_4.csv'

PCA_COMPONENTS = 0.8
CUTT_OFF_FREQUENCY = 2
ORDER = 4
SAMPLE_FREQUENCY = 500

EPOCHS = 5
WINDOW_SIZE = 500

In [22]:
def prepare_training_data(data_path):
    data = pd.read_csv(data_path)
    event_path = data_path.replace('_data', '_events')
    labels = pd.read_csv(event_path)
    clean_data = data.drop(['id'], axis = 1)
    labels = labels.drop(['id'], axis = 1)
    return clean_data, labels

In [23]:
def prepare_test_data(data_path):
    data = pd.read_csv(data_path)
    return data

In [24]:
def scaler_transform(data, scaler):
    if scaler == None:
        scaler = StandardScaler()
        return scaler.fit_transform(data), scaler
    else:
        return scaler.transform(data)        

In [25]:
def read_training_data(train_data_paths):
    labels_raw = []
    features_raw = []
    for data_path in train_data_paths:
        data, labels = prepare_training_data(data_path)
        features_raw.append(data)
        labels_raw.append(labels)
    features_raw = pd.concat(features_raw)
    labels_raw = pd.concat(labels_raw)
    x_train = np.asarray(features_raw.astype(float))
    y_train = np.asarray(labels_raw.astype(float))
    return x_train, y_train

In [26]:
def read_test_data(test_data_paths):
    test_features_raw = []
    ids = []
    for data_path in test_data_paths:
        data = prepare_test_data(data_path)
        test_features_raw.append(data)
        ids.append(np.array(data['id']))
    test_features_raw = pd.concat(test_features_raw)
    ids = np.concatenate(ids)
    test_features_raw = test_features_raw.drop(['id'], axis = 1)
    x_test = np.asarray(test_features_raw.astype(float))
    return x_test, ids

In [27]:
def metric_auc_score(predictions, y_test, with_plot):
    scores = []
    legend_text = []
    for i in range(N_LABELS):
        fpr, tpr, _  = roc_curve(y_test[:,i], predictions[:,i], 1)
        scores.append(roc_auc_score(y_test[:,i], predictions[:,i]))
        legend_text.append(COLUMNS[i]+' (area = %.3f)' % (scores[i]))
        if with_plot == True:
            plt.plot(fpr, tpr)
    if with_plot == True:
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curves')
        plt.legend(legend_text)
        plt.show()
    return scores

In [28]:
def single_metric_auc_score(predictions, y_test, with_plot):
    fpr, tpr, _  = roc_curve(y_test, predictions, 1)
    score = roc_auc_score(y_test, predictions)
    print(COLUMNS[0]+' AUC score = %.3f' % (score))
    if with_plot == True:
        plt.plot(fpr, tpr)
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curves')
        plt.show()
    return score

In [29]:
def all_auc_scores(prediction_total, test_data_total, subjects, with_plot):
    scores = []
    for subject in subjects:
        score = metric_auc_score(prediction_total[subject-1],
                                 test_data_total[subject-1], with_plot)
        scores.append(score)
        print('Mean AUC Score of Subject %d: %.3f' % \
              (subject, np.mean(score)))
    return scores

In [30]:
def image_mappping(x_train, WINDOW_SIZE):
    result = []
    empty_matrix = np.atleast_3d(np.zeros(np.shape(x_train[0:WINDOW_SIZE])))
    for i in range(len(x_train)):
        if i-WINDOW_SIZE < 0:
            result.append(empty_matrix)
        else:
            result.append(np.atleast_3d(x_train[i-WINDOW_SIZE:i]))
    return result

In [31]:
def train_generator(x_train, y_train, WINDOW_SIZE):
    x = image_mappping(x_train, WINDOW_SIZE)
    while True:
        for image, task in zip(x, y_train):
            yield np.array([image]), np.array([task])

In [32]:
def test_generator(x_test, WINDOW_SIZE):
    x = image_mappping(x_test, WINDOW_SIZE)
    while True:
        for image in x:
            yield np.array([image])

In [33]:
def class_weights(y_train):
    class_weight = class_weight.compute_class_weight('balanced', np.unique(y_train[:,0]), y_train[:,0])
    return {0 : 1.,
    1: class_weight[1],
    2: class_weight[0]}

In [34]:
def init_cnn(window):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=(window,32,1)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(4, 4)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(4, 4)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense((2)))
    model.add(Activation('softmax'))
    model.summary()
    optimizer = Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [35]:
for subject in SUBJECTS:
    test_features_raw = []
    train_data_paths = glob(TRAIN_DATA_PATH % (subject))
  
    x_train, y_train = read_training_data(train_data_paths)  
    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.5)
    
    x_train, scaler = scaler_transform(x_train, None)
    x_test = scaler_transform(x_test, scaler)
    
    train_labels = to_categorical(y_train, num_classes = None)
    test_labels = to_categorical(y_test, num_classes = None)

    class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train[:,0]), y_train[:,0])
    class_weights = {0 : 1., 1: class_weights[1], 2: class_weights[0]}
    
    


In [36]:
model = init_cnn(window=WINDOW_SIZE)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 500, 32, 32)       320       
_________________________________________________________________
activation_7 (Activation)    (None, 500, 32, 32)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 498, 30, 32)       9248      
_________________________________________________________________
activation_8 (Activation)    (None, 498, 30, 32)       0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 124, 7, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 124, 7, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 124, 7, 64)        18496     
__________

array([[[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       ...,

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]]])

In [40]:
my_generater = train_generator(x_train, train_labels, WINDOW_SIZE)
# model.fit_generator(train_generator, steps_per_epoch=len(x_train)/SUBSAMPLE, epochs=3, class_weight=class_weights, verbose=1)

In [ ]:
my_test_generater = test_generator(x_test, WINDOW_SIZE)
predictions = model.predict_generator(my_test_generater, steps=len(x_test)/SUBSAMPLE)

In [ ]:
score = single_metric_auc_score(predictions[:, 1], test_labels[:,0], True)
print('AUC score: %.3f' % (score))